In [1]:
import numpy as np
from scipy.integrate import quad

def toK(c):
    return (c + 273.15)

def toPa(bar):
    return (bar * 1e5)

def toBar(Pa):
    return (Pa / 1e5)
    
R = 8.314462618 # J/(K·mol)
R_Bar =  0.00008314462618 # bar·m³/(K·mol)
Tpad = 298.15 # temperatura padrão 25 Celcius
Ppad = 1 # Bar

class Comp: # Compostos : Classe que abriga as propriedades e funções
    def __init__(self, name, omega, Pc, Tb, Tc, NCParams, Ant = None, Hf = 0): # NCParams: A, B, C, D cp do V.Ness; Ant: A, B, C 
        self.name = name
        self.omega = omega
        self.Pc = Pc
        self.Tb = Tb
        self.Tc = Tc
        self.NCParams = NCParams
        self.Ant = Ant
        self.Hf = Hf

    def Antoine(self, T): # NIST, T em K, P em Bar
        AntA = self.Ant[0]
        AntB = self.Ant[1]
        AntC = self.Ant[2]
        if T < self.Ant[3] or T > self.Ant[4]:
            print(f"{self.name} : Temperatura fora da faixa para antoine")
        return 10 ** (AntA - (AntB / (T + AntC)))

    def TAntoine(self, P): # Eq antoine para temperatura, P em Bar
        Ant = self.Ant
        T = (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]
        return T

    def Estado(self, P, T): # Retorna o estado do composto
        if T > self.Tc and P > self.Pc:
            print(f"{self.name} : Fluído supercrítico")
            return "gas"
        elif T > self.Tc or self.Ant == None:
            return "gas"
        elif T < self.Tc and P > self.Pc:
            return "liquid"
        elif T < self.Tc and P < self.Pc:
            if  P < self.Antoine(T):
                return "vapour"
            elif P == self.Antoine(T):
                print(f"{self.name} : FODEO")
            else:
                return "liquid"

    def Cp_ig(self, T): # Equação 142 V.Ness
        A = self.NCParams[0]
        B =  self.NCParams[1]
        C = self.NCParams[2]
        D =  self.NCParams[3]
        return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))
    
    def SCp_ig(self, T): # equação p 188 V.Ness
        return self.Cp_ig(T) /(R * T)

    def Cp_L(self, T): # Cp de Rowlison Bondi
        return R * (1.45 + (0.45/(1-(T / self.Tc)) + 0.25 * self.omega * (17.11 + (25.2 * (1 - (T / self.Tc))**(1/3)) / (T / self.Tc) + (1.742)/(1 - (T / self.Tc)) ) ) ) + self.Cp_ig(T)
    
    def Hres(self, P, T): # Por Peng Robinson slide aula 2 integração
        P = toPa(P)
        k = 0.37464 + (0.154226 * self.omega) - (0.26992 * (self.omega ** 2))
        alphat = (1 + k * (1 - np.sqrt(T/self.Tc))) ** 2
        ac = 0.45724 * (((R ** 2) * (self.Tc ** 2)) / toPa(self.Pc))
        a = ac * alphat
        b = 0.07780 * R * (self.Tc / toPa(self.Pc))
        A = (a * P) / ((R ** 2) * (T ** 2.5))
        B =  (b * P) / (R * T)
        alpha = -1 + B
        beta = A - (3 * B ** 2 ) - 2 * B
        gamma = - (A * B) + (B ** 2) + (B ** 3) 
        Z = np.roots([1, alpha, beta, gamma]) # Resolução cúbica
        Z = Z[np.isreal(Z)] 
        Z = max(Z)
        Tdadt = - ac * k * alphat * (T / self.Tc)
        h = (b * P) / (Z * R * T)
        return R * T * (  (Z - 1) - ((a - Tdadt)/(2 * np.sqrt(2) * b * R * T)) * np.log((1 + h * (1 + np.sqrt(2)))/(1 + h * (1 - np.sqrt(2))))  )

    def Hvap(self): # Por Riedel p147 V.Ness
        return R * self.Tb * ((1.092 * (np.log(self.Pc) - 1.013)) / (0.930 - (self.Tb / self.Tc))) # P em bar mesmo

    def Hmolar(self, P, T): # Calcula a entalpia molar
        match self.Estado(Ppad, Tpad): # estado inicial
            case "gas" | "vapour":
                match self.Estado(Ppad, T): # estado final
                    case "gas" | "vapour":
                        print("caminho gás - gás")
                        H = quad(self.Cp_ig, Tpad, T)[0] + self.Hres(P, T)
                        print(f"{self.name} : {H} J/mol")
                        return H
                    case "liquid":
                        print("caminho gás - líquido")
                        Tvap = self.TAntoine(Ppad)
                        HGas = quad(self.Cp_ig, Tpad, Tvap)[0] + self.Hres(P, Tvap)
                        Hv = self.Hvap()
                        Hl = quad(self.Cp_L, Tvap, T)[0]
                        H = HGas - Hv + Hl # Menos Hvaporização já que é o processo inverso, condensação
                        print(f"{self.name} : {H} J/mol")
                        return H
            case "liquid":
                match self.Estado(Ppad, T): #estado final
                    case "gas" | "vapour":
                        print("caminho líquido - gás")
                        Tvap = self.TAntoine(Ppad)
                        # print(f"tvap = {Tvap}")
                        Hl = quad(self.Cp_L, Tpad, Tvap)[0]
                        # print(f"Hl = {Hl}")
                        Hv = self.Hvap()
                        # print(f"Hv = {Hv}")
                        HGas = quad(self.Cp_ig, Tvap, T)[0] + self.Hres(P, T)
                        # print(f"HGas = {HGas}")
                        H = Hl + Hv + HGas
                        print(f"{self.name} : {H} J/mol\n")
                        return H
                    case "liquid":
                        print("caminho líquido - líquido")
                        H = quad(self.Cp_L, Tpad, T)[0]
                        print(f"{self.name} : {H} J/mol")
                        return H

# Componentes

C2H4O = Comp( # 
        "Oxido de etileno",
        0.201, # omega Perry
        toBar(7.26e6), # Pc Perry
        toK(10.4), # Tb
        469.15, # Tc Perry
        [-0.385, # Cp A V.Ness
        23.463e-3, # B
        -9.296e-6, # C
        0], # D
        [5.84696, 2022.83, 62.656, 273.4, 304.9], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -52.630e-3) # Hf J/kmol  V. Ness

O2 = Comp( # 
        "Oxigênio",
        0.020, # omega Perry
        toBar(5.02e6), # Pc Perry
        toK(-182.953), # Tb
        154.58, # Tc Perry
        [3.639, # Cp A V.Ness
        0.506e-3, # B
        0, # C
        -0.227e5], # D
        # [3.9523, 340.024, --4.144, 54.36, 154.33], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 0) # Hf J/kmol 

N2 = Comp( #
        "Nitrogênio",
        0.037, # omega Perry
        toBar(3.39e6), # Pc Perry
        toK(-195.798), # Tb
        126.2, # Tc Perry
        [3.280, # Cp A V.Ness
        0.593e-3, # B
        0, # C
        0.040e5], # D
        # [3.7362, 264.651, -6.788, 63.14, 126], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 0) # Hf J/kmol 

C2H4 = Comp( # 
        "Etileno",
        0.086, # omega Perry
        toBar(5.03e6), # Pc Perry
        toK(-103.8), # Tb
        282.34, # Tc Perry
        [1.424, # Cp A V.Ness
        14.394e-3, # B
        -4.392, # C
        0], # D
        [13.871, 2854.7, -36.23, toK(-169.16), toK(76.52)], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 52.510e-3) # Hf J/kmol  V. Ness

CO2 = Comp( #
        "Dioxido de carbono",
        0.224, # omega Perry
        toBar(7.39e6), # Pc Perry
        toK(-78.464), # Tb
        304.21, # Tc Perry
        [5.457, # Cp A V.Ness 
        1.045e-3, # B 
        0, # C 
        -1.157e5], # D 
        # [6.81228, 1301.679, -3.494, 154.26, 195.89]   , # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -393.509e-3) # Hf V. Ness 
    
CH3CHO = Comp( # 
        "Acetaldeído",
        0.292, # omega Perry
        toBar(5.57e6), # Pc Perry
        toK(20.6), # Tb Haynes
        466, # Tc Perry
        [-0.385, # Cp A V.Ness
        23.463e-3, # B
        -9.296e-6, # C
        0], # D
        [3.68639, 822.894, -69.899, 293.4, 377.5], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -166.190e-3) # J/kmol V. Ness

H2O = Comp( # 
        "Água",
        0.343, # omega Perry
        toBar(21.94e6), # Pc Perry
        toK(99.974), # Tb Haynes
        647.13, # Tc Perry
        [3.470, # Cp A V.Ness
        1.450e-3, # B
        0, # C
        0.121e5], # D
        [5.40221, 1838.675, -31.737, 273, 303], # Antoine A, B, C, D, Tmin, Tmax
        Hf = -285.830e-3) # Líquido J/kmol Hf V. Ness


class Corrente: # Corrente : Classe que abriga uma corrente toda
    def __init__(self, name, P, T, C2H4O = 0, O2 = 0, N2 = 0, C2H4 = 0, CO2 = 0, H2O = 0, CH3CHO = 0): # pressão, temperatura e vazões molares kmol / h
        self.name =  name
        self.P = P
        self.T = T
        self.C2H4O = C2H4O
        self.O2 = O2
        self.N2 = N2
        self.C2H4 = C2H4
        self.CO2 = CO2
        self.H2O = H2O
        self.CH3CHO = CH3CHO
    def Hcorr(self):
        H = (self.C2H4O * C2H4O.Hmolar(self.P, self.T) + self.O2 * O2.Hmolar(self.P, self.T) + self.N2 * N2.Hmolar(self.P, self.T) 
             + self.C2H4 * C2H4.Hmolar(self.P, self.T) + self.CO2 * CO2.Hmolar(self.P, self.T) + self.H2O * H2O.Hmolar(self.P, self.T) + self.CH3CHO * CH3CHO.Hmolar(self.P, self.T))        
        print(f"{self.name} : {H} kJ/h")
        return H

In [ ]:
# TC-01
C3 = Corrente("Corrente 3, Entrada TC-01", 22, toK(30))
C4a = Corrente("Corrente 4a, Saída TC-01", 20, toK())
C5 = Corrente("Corrente 5, Entrada TC-01", 19, toK(240))
C5a = Corrente("Corrente 5a, Saída TC-01", 19, toK())

In [ ]:
# F-01
# QF1 = C4b - C4a
C4b = Corrente("Corrente 4b, Saída Forno", 20, toK(200))
QF1 = C4b.Hcor() - C4a.Hcor() 
print(f"F-01: {QF1} kJ/h")

In [ ]:
# R-01
# Reação lei de hess

R = 


In [ ]:
# TC-02
# QTC2 = C5b - C5a
C5b = Corrente("Corrente 5a, Saída TC-02", 18, toK(50))
QTC2 = C5b.Hcor() - C5a.Hcor() 
print(f"TC-02: {QTC2} kJ/h")

In [3]:
# Dest-01
# Roult na corrente de fundo
C12 = Corrente("Corrente 12, Entrada", 5.8, CH3CHO.TAntoine(5.8),1,0,0,0,0,0)
C15 = Corrente("Corrente 15, Topo", 5.4, C2H4O.TAntoine(5.8), )
C16 = Corrente("Corrente 16, Fundo", 5.6, H2O.TAntoine(5.6), ) # ALTERAR
C15c = Corrente("Corrente 15, Refluxo", 5.4, C2H4O.TAntoine(5.4), )

# C15c - C15 = TC-06
TC06 = C15c.Hcorr() - C15.Hcorr() 

# C15 + C16 - C15 - C15c = Q, Q = TC-06 + TC-07
TC07 = C15.Hcorr() + C16.Hcorr() - C15.Hcorr() - C15c.Hcorr() - TC06 

print(f"TC-06: {TC06} kJ/h, TC-07: {TC07} kJ/h")

Oxido de etileno : Temperatura fora da faixa para antoine
caminho gás - gás
Oxido de etileno : 5294.341068127421 J/mol
caminho gás - gás
Oxigênio : 2912.795743420723 J/mol
caminho gás - gás
Nitrogênio : 2831.4274064108818 J/mol
caminho gás - gás
Etileno : -427565301.9531246 J/mol
caminho gás - gás
Dioxido de carbono : 3880.674147792417 J/mol
Água : Temperatura fora da faixa para antoine
caminho líquido - gás
Água : 34734.50388827087 J/mol

Acetaldeído : Temperatura fora da faixa para antoine
caminho gás - gás
Acetaldeído : 5283.294372060465 J/mol
Corrente 12 : 5294.341068127421 kJ/h


5294.341068127421

In [3]:
# Para descoberta da temperatura Corrente 3
C2 = [
    0,
    0,
    265.6704877,
    60.1569231,
    1163.056436,
    15.03923078,
]  # Vazões CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5

T2 = toK(163)
P2 = 52
C12 = [
    6.24030213769374,
    2.41078129059267,
    213.428133248607,
    0,
    12251.7033530114,
    393.828789949421,
]  # Vazões CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5

T12 = toK(22)
P12 = 52
# C-2
C_2 = (
    C2[2] * CO.Hmolar(P2, T2)
    + C2[3] * CO2.Hmolar(P2, T2)
    + C2[4] * H2.Hmolar(P2, T2)
    + C2[5] * N2.Hmolar(P2, T2)
) # Entalpia da corrente
# C-12
C_12 = (
    C12[0] * CH3OH.Hmolar(P12, T12)
    + C12[1] * H2O.Hmolar(P12, T12)
    + C12[2] * CO.Hmolar(P12, T12)
    + C12[3] * CO2.Hmolar(P12, T12)
    + C12[4] * H2.Hmolar(P12, T12)
    + C12[5] * N2.Hmolar(P12, T12)
) # Entalpia da corrente
# print(f"{C_12} kJ / mol H")
# C3
C3 = [
    6.24030213769363,
    2.41078129059269,
    479.098620992281,
    60.1569231043921,
    13414.759788997,
    408.868020725519,
]  # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5
P3 = 52  # Estimativa
C_3 = C_2 + C_12
T3 = 0
for T in np.arange(T12, T2, 0.01):
    C3ite = (
        C3[0] * CH3OH.Hmolar(P3, T)
        + C3[1] * H2O.Hmolar(P3, T)
        + C3[2] * CO.Hmolar(P3, T)
        + C3[3] * CO2.Hmolar(P3, T)
        + C3[4] * H2.Hmolar(P3, T)
        + C3[5] * N2.Hmolar(P3, T)
    )
    # print(f"Dif:{abs(C_3 - C3ite)}, T:{T - 273.15}")
    if C3ite < C_3 + 1e4 and C3ite > C_3 - 1e4:
        print(T - 273.15)
        T3 = T
        break

# C-3
C_3 = ( 
    C3[0] * CH3OH.Hmolar(P3, T3)
    + C3[1] * H2O.Hmolar(P3, T3)
    + C3[2] * CO.Hmolar(P3, T3)
    + C3[3] * CO2.Hmolar(P3, T3)
    + C3[4] * H2.Hmolar(P3, T3)
    + C3[5] * N2.Hmolar(P3, T3)
)


NameError: name 'CO' is not defined

In [ ]:
# TC-1
# Tubos C3 -> C4
# C-4 C3 = C4 vazão igual
T4 = toK(220)
P4 = 51
C4 = C3
C_4 = (
    C4[0] * CH3OH.Hmolar(P4, T4)
    + C4[1] * H2O.Hmolar(P4, T4)
    + C4[2] * CO.Hmolar(P4, T4)
    + C4[3] * CO2.Hmolar(P4, T4)
    + C4[4] * H2.Hmolar(P4, T4)
    + C4[5] * N2.Hmolar(P4, T4)
) # Entalpia da corrente
# print(f'C3:{C_3}, C4: {C_4}, Diff: {C_4 - C_3}')
QTC1 = C_4 - C_3

# Casco C6 -> C7
T6 = toK(280)
P6 = 50
C6 = [
    323.912734025437,
    62.5677043949848,
    221.58311220893,
    0,
    12719.2580021172,
    408.868020725519,
] # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5

C_6 = (
    C6[0] * CH3OH.Hmolar(P6, T6)
    + C6[1] * H2O.Hmolar(P6, T6)
    + C6[2] * CO.Hmolar(P6, T6)
    + C6[3] * CO2.Hmolar(P6, T6)
    + C6[4] * H2.Hmolar(P6, T6)
    + C6[5] * N2.Hmolar(P6, T6)
) # Entalpia da corrente
C7 = C6
P7 = P6

# Qtubos = -Qcasco
for T in np.arange(T3, T6, 0.01):
    C7ite = (
        C7[0] * CH3OH.Hmolar(P7, T)
        + C7[1] * H2O.Hmolar(P7, T)
        + C7[2] * CO.Hmolar(P7, T)
        + C7[3] * CO2.Hmolar(P7, T)
        + C7[4] * H2.Hmolar(P7, T)
        + C7[5] * N2.Hmolar(P7, T)
    )
    QTC1ite = C7ite - C_6 
    # print(f"Diff:{abs(QTC1 + QTC1ite)}, T:{T - 273.15}")
    if QTC1 + QTC1ite < 1e5 and QTC1ite + QTC1 > -1e5:
        print(T - 273.15)
        # print(QTC1 + QTC1ite)
        T7 = T
        break
# Iterando 
while True:
    T7old = T7
    T4 = T7 - 20
    C_4 = (
        C4[0] * CH3OH.Hmolar(P4, T4)
        + C4[1] * H2O.Hmolar(P4, T4)
        + C4[2] * CO.Hmolar(P4, T4)
        + C4[3] * CO2.Hmolar(P4, T4)
        + C4[4] * H2.Hmolar(P4, T4)
        + C4[5] * N2.Hmolar(P4, T4)
    ) # Entalpia da corrente
    QTC1 = C_4 - C_3
    for T in np.arange(T3, T6, 0.01):
        C7ite = (
            C7[0] * CH3OH.Hmolar(P7, T)
            + C7[1] * H2O.Hmolar(P7, T)
            + C7[2] * CO.Hmolar(P7, T)
            + C7[3] * CO2.Hmolar(P7, T)
            + C7[4] * H2.Hmolar(P7, T)
            + C7[5] * N2.Hmolar(P7, T)
        )
        QTC1ite = C7ite - C_6 
        # print(f"Diff:{abs(QTC1 + QTC1ite)}, T:{T - 273.15}")
        if QTC1 + QTC1ite < 1e4 and QTC1ite + QTC1 > -1e4:
            # print(T - 273.15)
            # print(QTC1 + QTC1ite)
            T7 = T
            break
    if abs(T7old - T7) < 1:
        print(T7 - 273.15)
        break

In [ ]:
# Forno
# C-4
C_4 = (
    C4[0] * CH3OH.Hmolar(P4, T4)
    + C4[1] * H2O.Hmolar(P4, T4)
    + C4[2] * CO.Hmolar(P4, T4)
    + C4[3] * CO2.Hmolar(P4, T4)
    + C4[4] * H2.Hmolar(P4, T4)
    + C4[5] * N2.Hmolar(P4, T4)
)  # Entalpia da corrente
# C-5
C5 = C4
T5 = toK(240)
P5 = 51 

C_5 = (
    C5[0] * CH3OH.Hmolar(P5, T5)
    + C5[1] * H2O.Hmolar(P5, T5)
    + C5[2] * CO.Hmolar(P5, T5)
    + C5[3] * CO2.Hmolar(P5, T5)
    + C5[4] * H2.Hmolar(P5, T5)
    + C5[5] * N2.Hmolar(P5, T5)
)  # Entalpia da corrente

QF1 = C_5 - C_4
print(f"F-1: {QF1} kJ/(h)")



In [ ]:
# Reator
# Reação 1
xi1 = abs(C6[2] - C3[2]) # variação de CO é o grau de avanço da reação 1
hR1 = CH3OH.Hf - (CO.Hf) # entalpia molar J/mol H2 não tem Hf
HR1 = xi1 * hR1 # entalpia R1 kJ/molh
# Reação 2
xi2 = abs(C6[3] - C3[3]) # variação de CO2 é o grau de avanço da reação 2
hR2 = CH3OH.Hf + H2O.Hf - (CO2.Hf) # entalpia molar J/mol H2 não tem Hf
HR2 = xi2 * hR2 # entalpia R2 kJ/molh

print(f"hmR1:{hR1} J/mol, R1: {HR1} kJ/h, hmR2: {hR2} J/mol, R2: {HR2} kJ/h, DH:{ C_6 - C_5} kJ/h")

Qr = C_6 - C_5 + HR1 + HR2
print(f"QR: {Qr} kJ/h")

In [ ]:
# TC-2

C_7 = (
    C7[0] * CH3OH.Hmolar(P7, T7)
    + C7[1] * H2O.Hmolar(P7, T7)
    + C7[2] * CO.Hmolar(P7, T7)
    + C7[3] * CO2.Hmolar(P7, T7)
    + C7[4] * H2.Hmolar(P7, T7)
    + C7[5] * N2.Hmolar(P7, T7)
) # entalpia da corrente 7

# C8
C8 = C7
T8 = toK(4)
P8 = 48
C_8 = (
    C8[0] * CH3OH.Hmolar(P8, T8)
    + C8[1] * H2O.Hmolar(P8, T8)
    + C8[2] * CO.Hmolar(P8, T8)
    + C8[3] * CO2.Hmolar(P8, T8)
    + C8[4] * H2.Hmolar(P8, T8)
    + C8[5] * N2.Hmolar(P8, T8)
) # entalpia da corrente 8

QTC2 = C_8 - C_7
print(f"QTC2: {QTC2} kJ/h")

In [ ]:
# DE-1
# C16
C16 = [
    315.8473069,
    58.86369629,
    0.015004089,
    0,
    0.356347123,
    0.019880418,
]  # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5
T16 = toK(21)
P16 = 21
C_16 = (
    C16[0] * CH3OH.Hmolar(P16, T16)
    + C16[1] * H2O.Hmolar(P16, T16)
    + C16[2] * CO.Hmolar(P16, T16)
    + C16[3] * CO2.Hmolar(P16, T16)
    + C16[4] * H2.Hmolar(P16, T16)
    + C16[5] * N2.Hmolar(P16, T16)
)  # entalpia da corrente 16
# C17
C17 = [
    624.2199832,
    0.624844828,
    0.015004089,
    0,
    0.356347123,
    0.019880418,
]  # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5
P17 = 2.8
T17 = toK(123)
C_17 = (
    C17[0] * CH3OH.Hmolar(P17, T17)
    + C17[1] * H2O.Hmolar(P17, T17)
    + C17[2] * CO.Hmolar(P17, T17)
    + C17[3] * CO2.Hmolar(P17, T17)
    + C17[4] * H2.Hmolar(P17, T17)
    + C17[5] * N2.Hmolar(P17, T17)
)  # entalpia da corrente 17
# C18
C18 = C17  # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5
P18 = 2.8
T18 = CH3OH.TAntoine(2.8)
C_18 = (
    C18[0] * CH3OH.Hmolar(P18, T18)
    + C18[1] * H2O.Hmolar(P18, T18)
    + C18[2] * CO.Hmolar(P18, T18)
    + C18[3] * CO2.Hmolar(P18, T18)
    + C18[4] * H2.Hmolar(P18, T18)
    + C18[5] * N2.Hmolar(P18, T18)
)  # entalpia da corrente 17
# C21
C21 = [
    312.1099916,
    0.312422414,
    0,
    0,
    0,
    0,
]  # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5
P21 = 2.8
T21 = CH3OH.TAntoine(2.8)
C_21 = (
    C21[0] * CH3OH.Hmolar(P21, T21)
    + C21[1] * H2O.Hmolar(P21, T21)
    + C21[2] * CO.Hmolar(P21, T21)
    + C21[3] * CO2.Hmolar(P21, T21)
    + C21[4] * H2.Hmolar(P21, T21)
    + C21[5] * N2.Hmolar(P21, T21)
)  # entalpia da corrente 21
# C24
C24 = [
    3.737315354,
    58.55127388,
    0,
    0,
    0,
    0,
]  # Vazões molares  CH3OH 0, H2O 1, CO 2, CO2 3, H2 4, N2 5
P24 = 2.9
T24 = 123
C_24 = np.real( #por algum motivo obscuro aparece com parcela imaginaria nula
    C24[0] * CH3OH.Hmolar(P24, T24)
    + C24[1] * H2O.Hmolar(P24, T24)
    + C24[2] * CO.Hmolar(P24, T24)
    + C24[3] * CO2.Hmolar(P24, T24)
    + C24[4] * H2.Hmolar(P24, T24)
    + C24[5] * N2.Hmolar(P24, T24)
)  # entalpia da corrente 24
print(f"c17 {C_17}, c24 {C_24}, c21 {C_21}, c16 {C_16}")
QtR = np.sum(C_17 + C_24 - C21 - C16) # soma usual do python não retorna o esperado
print(f"QtR: {QtR} kJ/h")
TC4 = C_18 - C_17
TC5 = QtR - TC4
print(f"TC4: {TC4} kJ/h, TC5: {TC5} kJ/h")

In [ ]:
# Utilidades
# F-1 

print(f"steam@ {3658.3e3 / 1e5} Bar, T: 260C , Hvap : 1739.5830023828435 kJ/kg, {QF1 / 1739.5830023828435 } kg/h")
# R
ares = H2O.Hmolar(1, toK(40)) #água resfriamento
print(f"H água {ares} J/mol @40C,1bar, Qre = {Qr} kJ/h, {Qr / ares } kmol/h, {(Qr / ares) * 18 } kg/h")


# TC-2
NH4 = Comp(0.253, 113.53, 239.82, 405.40, [3.578, 3.020e-3, 0, -0.186], [3.18757,506.713, -80.78, 164.0, 239.6], [4.86886, 1113.928, -10.409, 239.6, 371.5])
Resf = NH4.Hvap()
Pres = NH4.Antoine(toK(4))
print(f"P corrente {Pres} Bar, hvap = {Resf}, TC2 = {QTC2} kJ/h, {-QTC2 / Resf } kmol/h, {(-QTC2 / Resf) * 17 } kg/h")

# TC-4
print(f"H água {ares} J/mol @40C,1bar, TC4 = {-TC4} kJ/h, {-TC4 / ares } kmol/h, {(-TC4 / ares) * 18 } kg/h")

# TC-5
print(f"steam@ {3658.3e3 / 1e5} Bar, T: 260C , Hvap : 1739.5830023828435 kJ/kg, {TC5 / 1739.5830023828435 } kg/h")

In [ ]:
# Compressor

C1 = C2
P1 = 40
T1 = toK(120)
C_1 = (
    C1[0] * CH3OH.Hmolar(P1, T1)
    + C1[1] * H2O.Hmolar(P1, T1)
    + C1[2] * CO.Hmolar(P1, T1)
    + C1[3] * CO2.Hmolar(P1, T1)
    + C1[4] * H2.Hmolar(P1, T1)
    + C1[5] * N2.Hmolar(P1, T1)
)  # entalpia da corrente 1

WsC = C_2 - C_1 # Trabalho de eixo compressor 1
Tisen = 0
PPo = np.log(P2 / P1) # termo ln(P/Po)
print(PPo)
for T in np.arange(T1, T2, 0.01):
    est = quad(H2.SCp_ig, T1, T)[0] # H2 é o que está mais em abundancia na corrente
    if est < PPo + 1e-3 and est > PPo - 1e-3:
        print(T - 273.15)
        Tisen = T
        break
C_2isen = (
    C2[0] * CH3OH.Hmolar(P2, Tisen)
    + C2[1] * H2O.Hmolar(P2, Tisen)
    + C2[2] * CO.Hmolar(P2, Tisen)
    + C2[3] * CO2.Hmolar(P2, Tisen)
    + C2[4] * H2.Hmolar(P2, Tisen)
    + C2[5] * N2.Hmolar(P2, Tisen)
)  # entalpia da corrente 2 isoentropica

WsCisen = C_2isen - C_1 # Trabalho de eixo compressor 1 isoentropico

n = WsCisen / WsC
print(n)

In [ ]:
print(f"{ CH3OH.Hmolar(P2, Tisen):.2f}, { H2O.Hmolar(P2, Tisen):.2f}, { CO.Hmolar(P2, Tisen):.2f}, { CO2.Hmolar(P2, Tisen):.2f}, { H2.Hmolar(P2, Tisen):.2f}, { N2.Hmolar(P2, Tisen):.2f}")
print(f"{(C1[0] * CH3OH.Hmolar(P2, Tisen)):.2f}, {(C1[1] * H2O.Hmolar(P2, Tisen)):.2f}, {(C1[2] * CO.Hmolar(P2, Tisen)):.2f}, {(C1[3] * CO2.Hmolar(P2, Tisen)):.2f}, {(C1[4] * H2.Hmolar(P2, Tisen)):.2f}, {(C1[5] * N2.Hmolar(P2, Tisen)):.2f}")
print(f"{WsCisen:.2f}")